# Setting

In [ ]:
import os

import numpy as np
import pandas as pd

from bionlp.util import fs, io, func


EVNT_ARG_TYPE = {'2016':{'Lives_In':['Bacteria', 'Location']}, '2013':{'Localization':['Bacterium', 'Localization'], 'PartOf':['Host', 'Part']}, '2011':{'RegulonDependence':['Regulon', 'Target'], 'BindTo':['Agent', 'Target'], 'TranscriptionFrom':['Transcription', 'Site'], 'RegulonMember':['Regulon', 'Member'], 'SiteOf':['Site', 'Entity'], 'TranscriptionBy':['Transcription', 'Agent'], 'PromoterOf':['Promoter', 'Gene'], 'PromoterDependence':['Promoter', 'Protein'], 'ActionTarget':['Action', 'Target'], 'Interaction':['Agent', 'Target']}}
FIX_EVNT_ARG_TYPE = {'2016':{'Lives_In':['Bacteria', 'Location']}, '2013':{'Localization':['Bacterium', 'Localization'], 'PartOf':['Host', 'Part']}, '2011':{'RegulonDependence':['Agent', 'Target'], 'BindTo':['Agent', 'Target'], 'TranscriptionFrom':['Process', 'Promoter'], 'RegulonMember':['Regulon', 'Member'], 'SiteOf':['Target', 'Site'], 'TranscriptionBy':['Transcription', 'Polymerase'], 'PromoterOf':['Agent', 'Target'], 'PromoterDependence':['Agent', 'Target'], 'ActionTarget':['Action', 'Target'], 'Interaction':['Agent', 'Target']}}

YEAR='2013'
TASK='bb'
DATA_PATH = '../../data/bioevent/bnlpst/%s/%s' % (YEAR, TASK)
TEST_DIR = os.path.join(DATA_PATH, 'dev')
PRED_DIR = os.path.join(DATA_PATH, 'pred')
FIX_PRED_DIR = os.path.join(DATA_PATH, 'pred.fix')
STAT_OA_FNAME = 'train_stat_oprnd_annot.pkl'

In [ ]:
stat_oprnd_annot = io.read_obj(os.path.join(DATA_PATH, STAT_OA_FNAME))

In [ ]:
stat_oprnd_annot

In [ ]:
fs.mkdir(FIX_PRED_DIR)
for fname in fs.listf(PRED_DIR, pattern='.*\.a2'):
    ref_lines = [l for l in fs.read_file(os.path.join(TEST_DIR, fname.replace('a2', 'a1'))) if l.startswith('T')]
    refs = {}
    for ref_line in ref_lines:
        ref_id, ref_data = ref_line.split('\t')[:2]
        refs[ref_id] = ref_data.split(' ')[0]

    new_idx, fix_lines = 1, []
    lines = fs.read_file(os.path.join(PRED_DIR, fname))
    for line in lines:
        ldata = line.split('\t')
        evnt_id = ldata[0]
        evnt_data = ldata[1].split(' ')
        evnt_type = evnt_data[0]
        orig_argtype = EVNT_ARG_TYPE[YEAR][evnt_type]
        fix_argtype = FIX_EVNT_ARG_TYPE[YEAR][evnt_type]
        for oat, fat, arg_idx in zip(orig_argtype, fix_argtype, range(1, len(evnt_data))):
            if (not stat_oprnd_annot[evnt_type][arg_idx - 1].has_key(refs[evnt_data[arg_idx].split(':')[1].strip()])): break
            evnt_data[arg_idx] = evnt_data[arg_idx].replace(oat, fat)
        else:
            fix_lines.append('E%i\t%s' % (new_idx, ' '.join(evnt_data)))
            new_idx += 1
    fs.write_file(''.join(fix_lines), os.path.join(FIX_PRED_DIR, fname))